# Overview

This notebook creates the network dataset and runs the service area analysis.

# Setup

In [45]:
# Import libraries
import arcpy, pandas as pd, numpy as np, os

In [61]:
# Set workspace settings
arcpy.env.workspace  = r"C:\Users\kathr\Documents\outdoor-alliance\mt-hood\mt-hood-analysis\mh_output.gdb"
arcpy.env.overwriteOutput = True

In [47]:
# Set path to gdb
gdb_path =  r"C:\Users\kathr\Documents\outdoor-alliance\mt-hood\mt-hood-analysis\mh_output.gdb"

# Create Network Dataset

In [48]:
nd_location = os.path.join(gdb_path, "mh_roads") # target feature dataset, must contain "all_roads_merge"
template = r"C:\Users\kathr\Documents\outdoor-alliance\san-gabriel\nd_template.xml" # manually generated template with network settings
nd_name = "network_dataset" # set by the template - don't change
nd_path = os.path.join(nd_location, nd_name)

In [49]:
# Create network dataset from template
# This requires the feature class "all_roads_merge" to be present in the target feature dataset
if not arcpy.Exists(nd_path):
    arcpy.nax.CreateNetworkDatasetFromTemplate(network_dataset_template = template,
                                               output_feature_dataset = nd_location)

In [50]:
# Build the network dataset
arcpy.nax.BuildNetwork(nd_path)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\mt-hood\\mt-hood-analysis\\mh_output.gdb\\mh_roads\\network_dataset'>

# Service Area Analysis

In [51]:
# Create network dataset layer - improves performance (rather than referencing network dataset multiple times)
nd_layer_name = "Mt_Hood_ND"
arcpy.nax.MakeNetworkDatasetLayer(nd_path, nd_layer_name)

<Result 'Mt_Hood_ND'>

In [52]:
# Instantiate a ServiceArea analysis object
service_area = arcpy.nax.ServiceArea(nd_layer_name)

## Rings

In [53]:
# Set properties for the analysis
# Available properties: https://pro.arcgis.com/en/pro-app/latest/arcpy/network-analyst/servicearea.htm

service_area.travelMode = arcpy.nax.GetTravelModes(nd_layer_name)["Automobile"] # distance-based
service_area.distanceUnits = arcpy.nax.DistanceUnits.Miles
service_area.defaultImpedanceCutoffs = [5, 30, 90]
service_area.geometryAtCutoff = arcpy.nax.ServiceAreaPolygonCutoffGeometry.Rings # outer service areas do not include inner
service_area.outputType = arcpy.nax.ServiceAreaOutputType.Polygons
service_area.geometryAtOverlap = arcpy.nax.ServiceAreaOverlapGeometry.Dissolve
service_area.polygonDetail = arcpy.nax.ServiceAreaPolygonDetail.High

In [54]:
# Load facilities
input_facilities = os.path.join(gdb_path, "visitor_centers")
service_area.load(arcpy.nax.ServiceAreaInputDataType.Facilities, input_facilities)

In [55]:
# Solve the analysis
result = service_area.solve()

In [56]:
# Export the results to a feature class. If the analysis failed print all the messages.
output_feature_class = os.path.join(gdb_path, "nd_output_rings")
if result.solveSucceeded:
    result.export(arcpy.nax.ServiceAreaOutputDataType.Polygons, output_feature_class)
else:
    arcpy.AddError("Analysis failed")
    # Print all the warning messages.
    for message in result.solverMessages(arcpy.nax.MessageSeverity.Warning):
        arcpy.AddWarning(message[-1])
    # Print all the error messages.
    for message in result.solverMessages(arcpy.nax.MessageSeverity.Error):
        arcpy.AddError(message[-1])

## Disks

In [57]:
# Set properties for the analysis
# Available properties: https://pro.arcgis.com/en/pro-app/latest/arcpy/network-analyst/servicearea.htm

service_area.travelMode = arcpy.nax.GetTravelModes(nd_layer_name)["Automobile"] # distance-based
service_area.distanceUnits = arcpy.nax.DistanceUnits.Miles
service_area.defaultImpedanceCutoffs = [5, 30, 90]
service_area.geometryAtCutoff = arcpy.nax.ServiceAreaPolygonCutoffGeometry.Disks # outer service areas do include inner
service_area.outputType = arcpy.nax.ServiceAreaOutputType.Polygons
service_area.geometryAtOverlap = arcpy.nax.ServiceAreaOverlapGeometry.Dissolve
service_area.polygonDetail = arcpy.nax.ServiceAreaPolygonDetail.High

In [58]:
# Re-load facilities
service_area.load(arcpy.nax.ServiceAreaInputDataType.Facilities, input_facilities)

In [59]:
# Re-solve the analysis
result = service_area.solve()

In [63]:
# Re-export the results to a feature class. If the analysis failed print all the messages.
output_feature_class = os.path.join(gdb_path, "nd_output_disks")
if result.solveSucceeded:
    result.export(arcpy.nax.ServiceAreaOutputDataType.Polygons, output_feature_class)
else:
    arcpy.AddError("Analysis failed")
    # Print all the warning messages.
    for message in result.solverMessages(arcpy.nax.MessageSeverity.Warning):
        arcpy.AddWarning(message[-1])
    # Print all the error messages.
    for message in result.solverMessages(arcpy.nax.MessageSeverity.Error):
        arcpy.AddError(message[-1])